In [1]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

# from config import db_password

import time

In [2]:
# Read in the country_vaccinations CSV file as Pandas DataFrames.
gdp_raw_data = pd.read_csv('../Resources/country_gdp.csv', low_memory=False)
gdp_raw_data


,Country Name,Population_2019,iso_code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Aruba,106314,ABW,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,...,2.549721e+09,2.534637e+09,2.701676e+09,2.765363e+09,2.919553e+09,2.965922e+09,3.056425e+09,NaN,NaN,NaN
1,Afghanistan,38041754,AFG,GDP (current US$),NY.GDP.MKTP.CD,5.377778e+08,5.488889e+08,5.466667e+08,7.511112e+08,8.000000e+08,...,1.780429e+10,2.000160e+10,2.056107e+10,2.048489e+10,1.990711e+10,1.801775e+10,1.886995e+10,1.835388e+10,1.929110e+10,NaN
2,Angola,31825295,AGO,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,...,1.120000e+11,1.280000e+11,1.370000e+11,1.460000e+11,1.160000e+11,1.010000e+11,1.220000e+11,1.010000e+11,8.881570e+10,NaN
3,Albania,2854191,ALB,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,...,1.289077e+10,1.231983e+10,1.277622e+10,1.322814e+10,1.138685e+10,1.186120e+10,1.301969e+10,1.514702e+10,1.527918e+10,NaN
4,Andorra,77142,AND,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,...,3.629204e+09,3.188809e+09,3.193704e+09,3.271808e+09,2.789870e+09,2.896679e+09,3.000181e+09,3.218316e+09,3.154058e+09,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,Kosovo,1794248,XKX,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,...,6.701698e+09,6.499807e+09,7.074778e+09,7.396705e+09,6.442916e+09,6.719172e+09,7.245707e+09,7.942962e+09,7.926134e+09,NaN
260,"Yemen, Rep.",29161922,YEM,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,...,3.272642e+10,3.540134e+10,4.041524e+10,4.320647e+10,4.245062e+10,3.093598e+10,2.673614e+10,2.348627e+10,2.258108e+10,NaN
261,South Africa,58558270,ZAF,GDP (current US$),NY.GDP.MKTP.CD,7.575397e+09,7.972997e+09,8.497997e+09,9.423396e+09,1.037400e+10,...,4.160000e+11,3.960000e+11,3.670000e+11,3.510000e+11,3.180000e+11,2.960000e+11,3.500000e+11,3.680000e+11,3.510000e+11,NaN
262,Zambia,17861030,ZMB,GDP (current US$),NY.GDP.MKTP.CD,7.130000e+08,6.962857e+08,6.931429e+08,7.187143e+08,8.394286e+08,...,2.345952e+10,2.550306e+10,2.804555e+10,2.715073e+10,2.124334e+10,2.095475e+10,2.586817e+10,2.631214e+10,2.330977e+10,NaN


In [3]:
# Fill the Nan values with 0

gdp_clean_data = gdp_raw_data.fillna(0)
gdp_clean_data

,Country Name,Population_2019,iso_code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Aruba,106314,ABW,GDP (current US$),NY.GDP.MKTP.CD,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,2.549721e+09,2.534637e+09,2.701676e+09,2.765363e+09,2.919553e+09,2.965922e+09,3.056425e+09,0.000000e+00,0.000000e+00,0.0
1,Afghanistan,38041754,AFG,GDP (current US$),NY.GDP.MKTP.CD,5.377778e+08,5.488889e+08,5.466667e+08,7.511112e+08,8.000000e+08,...,1.780429e+10,2.000160e+10,2.056107e+10,2.048489e+10,1.990711e+10,1.801775e+10,1.886995e+10,1.835388e+10,1.929110e+10,0.0
2,Angola,31825295,AGO,GDP (current US$),NY.GDP.MKTP.CD,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,1.120000e+11,1.280000e+11,1.370000e+11,1.460000e+11,1.160000e+11,1.010000e+11,1.220000e+11,1.010000e+11,8.881570e+10,0.0
3,Albania,2854191,ALB,GDP (current US$),NY.GDP.MKTP.CD,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,1.289077e+10,1.231983e+10,1.277622e+10,1.322814e+10,1.138685e+10,1.186120e+10,1.301969e+10,1.514702e+10,1.527918e+10,0.0
4,Andorra,77142,AND,GDP (current US$),NY.GDP.MKTP.CD,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,3.629204e+09,3.188809e+09,3.193704e+09,3.271808e+09,2.789870e+09,2.896679e+09,3.000181e+09,3.218316e+09,3.154058e+09,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,Kosovo,1794248,XKX,GDP (current US$),NY.GDP.MKTP.CD,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,6.701698e+09,6.499807e+09,7.074778e+09,7.396705e+09,6.442916e+09,6.719172e+09,7.245707e+09,7.942962e+09,7.926134e+09,0.0
260,"Yemen, Rep.",29161922,YEM,GDP (current US$),NY.GDP.MKTP.CD,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,3.272642e+10,3.540134e+10,4.041524e+10,4.320647e+10,4.245062e+10,3.093598e+10,2.673614e+10,2.348627e+10,2.258108e+10,0.0
261,South Africa,58558270,ZAF,GDP (current US$),NY.GDP.MKTP.CD,7.575397e+09,7.972997e+09,8.497997e+09,9.423396e+09,1.037400e+10,...,4.160000e+11,3.960000e+11,3.670000e+11,3.510000e+11,3.180000e+11,2.960000e+11,3.500000e+11,3.680000e+11,3.510000e+11,0.0
262,Zambia,17861030,ZMB,GDP (current US$),NY.GDP.MKTP.CD,7.130000e+08,6.962857e+08,6.931429e+08,7.187143e+08,8.394286e+08,...,2.345952e+10,2.550306e+10,2.804555e+10,2.715073e+10,2.124334e+10,2.095475e+10,2.586817e+10,2.631214e+10,2.330977e+10,0.0


In [4]:
# Change column name to match the SQL schema

gdp_clean_data=gdp_clean_data.rename({'Country Name' : 'country_name'}, axis =1)
gdp_clean_data

,country_name,Population_2019,iso_code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Aruba,106314,ABW,GDP (current US$),NY.GDP.MKTP.CD,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,2.549721e+09,2.534637e+09,2.701676e+09,2.765363e+09,2.919553e+09,2.965922e+09,3.056425e+09,0.000000e+00,0.000000e+00,0.0
1,Afghanistan,38041754,AFG,GDP (current US$),NY.GDP.MKTP.CD,5.377778e+08,5.488889e+08,5.466667e+08,7.511112e+08,8.000000e+08,...,1.780429e+10,2.000160e+10,2.056107e+10,2.048489e+10,1.990711e+10,1.801775e+10,1.886995e+10,1.835388e+10,1.929110e+10,0.0
2,Angola,31825295,AGO,GDP (current US$),NY.GDP.MKTP.CD,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,1.120000e+11,1.280000e+11,1.370000e+11,1.460000e+11,1.160000e+11,1.010000e+11,1.220000e+11,1.010000e+11,8.881570e+10,0.0
3,Albania,2854191,ALB,GDP (current US$),NY.GDP.MKTP.CD,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,1.289077e+10,1.231983e+10,1.277622e+10,1.322814e+10,1.138685e+10,1.186120e+10,1.301969e+10,1.514702e+10,1.527918e+10,0.0
4,Andorra,77142,AND,GDP (current US$),NY.GDP.MKTP.CD,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,3.629204e+09,3.188809e+09,3.193704e+09,3.271808e+09,2.789870e+09,2.896679e+09,3.000181e+09,3.218316e+09,3.154058e+09,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,Kosovo,1794248,XKX,GDP (current US$),NY.GDP.MKTP.CD,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,6.701698e+09,6.499807e+09,7.074778e+09,7.396705e+09,6.442916e+09,6.719172e+09,7.245707e+09,7.942962e+09,7.926134e+09,0.0
260,"Yemen, Rep.",29161922,YEM,GDP (current US$),NY.GDP.MKTP.CD,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,3.272642e+10,3.540134e+10,4.041524e+10,4.320647e+10,4.245062e+10,3.093598e+10,2.673614e+10,2.348627e+10,2.258108e+10,0.0
261,South Africa,58558270,ZAF,GDP (current US$),NY.GDP.MKTP.CD,7.575397e+09,7.972997e+09,8.497997e+09,9.423396e+09,1.037400e+10,...,4.160000e+11,3.960000e+11,3.670000e+11,3.510000e+11,3.180000e+11,2.960000e+11,3.500000e+11,3.680000e+11,3.510000e+11,0.0
262,Zambia,17861030,ZMB,GDP (current US$),NY.GDP.MKTP.CD,7.130000e+08,6.962857e+08,6.931429e+08,7.187143e+08,8.394286e+08,...,2.345952e+10,2.550306e+10,2.804555e+10,2.715073e+10,2.124334e+10,2.095475e+10,2.586817e+10,2.631214e+10,2.330977e+10,0.0


In [5]:
# Format column names

gdp_clean_data = gdp_clean_data.rename({'Indicator Name' : 'indicator_name'}, axis =1)
gdp_clean_data

,country_name,Population_2019,iso_code,indicator_name,Indicator Code,1960,1961,1962,1963,1964,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Aruba,106314,ABW,GDP (current US$),NY.GDP.MKTP.CD,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,2.549721e+09,2.534637e+09,2.701676e+09,2.765363e+09,2.919553e+09,2.965922e+09,3.056425e+09,0.000000e+00,0.000000e+00,0.0
1,Afghanistan,38041754,AFG,GDP (current US$),NY.GDP.MKTP.CD,5.377778e+08,5.488889e+08,5.466667e+08,7.511112e+08,8.000000e+08,...,1.780429e+10,2.000160e+10,2.056107e+10,2.048489e+10,1.990711e+10,1.801775e+10,1.886995e+10,1.835388e+10,1.929110e+10,0.0
2,Angola,31825295,AGO,GDP (current US$),NY.GDP.MKTP.CD,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,1.120000e+11,1.280000e+11,1.370000e+11,1.460000e+11,1.160000e+11,1.010000e+11,1.220000e+11,1.010000e+11,8.881570e+10,0.0
3,Albania,2854191,ALB,GDP (current US$),NY.GDP.MKTP.CD,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,1.289077e+10,1.231983e+10,1.277622e+10,1.322814e+10,1.138685e+10,1.186120e+10,1.301969e+10,1.514702e+10,1.527918e+10,0.0
4,Andorra,77142,AND,GDP (current US$),NY.GDP.MKTP.CD,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,3.629204e+09,3.188809e+09,3.193704e+09,3.271808e+09,2.789870e+09,2.896679e+09,3.000181e+09,3.218316e+09,3.154058e+09,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,Kosovo,1794248,XKX,GDP (current US$),NY.GDP.MKTP.CD,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,6.701698e+09,6.499807e+09,7.074778e+09,7.396705e+09,6.442916e+09,6.719172e+09,7.245707e+09,7.942962e+09,7.926134e+09,0.0
260,"Yemen, Rep.",29161922,YEM,GDP (current US$),NY.GDP.MKTP.CD,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,3.272642e+10,3.540134e+10,4.041524e+10,4.320647e+10,4.245062e+10,3.093598e+10,2.673614e+10,2.348627e+10,2.258108e+10,0.0
261,South Africa,58558270,ZAF,GDP (current US$),NY.GDP.MKTP.CD,7.575397e+09,7.972997e+09,8.497997e+09,9.423396e+09,1.037400e+10,...,4.160000e+11,3.960000e+11,3.670000e+11,3.510000e+11,3.180000e+11,2.960000e+11,3.500000e+11,3.680000e+11,3.510000e+11,0.0
262,Zambia,17861030,ZMB,GDP (current US$),NY.GDP.MKTP.CD,7.130000e+08,6.962857e+08,6.931429e+08,7.187143e+08,8.394286e+08,...,2.345952e+10,2.550306e+10,2.804555e+10,2.715073e+10,2.124334e+10,2.095475e+10,2.586817e+10,2.631214e+10,2.330977e+10,0.0


In [6]:
# Format column names

gdp_clean_data = gdp_clean_data.rename({'Indicator Code' : 'indicator_code'}, axis =1)
gdp_clean_data

,country_name,Population_2019,iso_code,indicator_name,indicator_code,1960,1961,1962,1963,1964,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Aruba,106314,ABW,GDP (current US$),NY.GDP.MKTP.CD,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,2.549721e+09,2.534637e+09,2.701676e+09,2.765363e+09,2.919553e+09,2.965922e+09,3.056425e+09,0.000000e+00,0.000000e+00,0.0
1,Afghanistan,38041754,AFG,GDP (current US$),NY.GDP.MKTP.CD,5.377778e+08,5.488889e+08,5.466667e+08,7.511112e+08,8.000000e+08,...,1.780429e+10,2.000160e+10,2.056107e+10,2.048489e+10,1.990711e+10,1.801775e+10,1.886995e+10,1.835388e+10,1.929110e+10,0.0
2,Angola,31825295,AGO,GDP (current US$),NY.GDP.MKTP.CD,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,1.120000e+11,1.280000e+11,1.370000e+11,1.460000e+11,1.160000e+11,1.010000e+11,1.220000e+11,1.010000e+11,8.881570e+10,0.0
3,Albania,2854191,ALB,GDP (current US$),NY.GDP.MKTP.CD,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,1.289077e+10,1.231983e+10,1.277622e+10,1.322814e+10,1.138685e+10,1.186120e+10,1.301969e+10,1.514702e+10,1.527918e+10,0.0
4,Andorra,77142,AND,GDP (current US$),NY.GDP.MKTP.CD,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,3.629204e+09,3.188809e+09,3.193704e+09,3.271808e+09,2.789870e+09,2.896679e+09,3.000181e+09,3.218316e+09,3.154058e+09,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,Kosovo,1794248,XKX,GDP (current US$),NY.GDP.MKTP.CD,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,6.701698e+09,6.499807e+09,7.074778e+09,7.396705e+09,6.442916e+09,6.719172e+09,7.245707e+09,7.942962e+09,7.926134e+09,0.0
260,"Yemen, Rep.",29161922,YEM,GDP (current US$),NY.GDP.MKTP.CD,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,3.272642e+10,3.540134e+10,4.041524e+10,4.320647e+10,4.245062e+10,3.093598e+10,2.673614e+10,2.348627e+10,2.258108e+10,0.0
261,South Africa,58558270,ZAF,GDP (current US$),NY.GDP.MKTP.CD,7.575397e+09,7.972997e+09,8.497997e+09,9.423396e+09,1.037400e+10,...,4.160000e+11,3.960000e+11,3.670000e+11,3.510000e+11,3.180000e+11,2.960000e+11,3.500000e+11,3.680000e+11,3.510000e+11,0.0
262,Zambia,17861030,ZMB,GDP (current US$),NY.GDP.MKTP.CD,7.130000e+08,6.962857e+08,6.931429e+08,7.187143e+08,8.394286e+08,...,2.345952e+10,2.550306e+10,2.804555e+10,2.715073e+10,2.124334e+10,2.095475e+10,2.586817e+10,2.631214e+10,2.330977e+10,0.0


In [7]:
# Format column names

gdp_clean_data = gdp_clean_data.rename({'2019' : 'GDP_2019'}, axis =1)
gdp_clean_data

,country_name,Population_2019,iso_code,indicator_name,indicator_code,1960,1961,1962,1963,1964,...,2011,2012,2013,2014,2015,2016,2017,2018,GDP_2019,2020
0,Aruba,106314,ABW,GDP (current US$),NY.GDP.MKTP.CD,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,2.549721e+09,2.534637e+09,2.701676e+09,2.765363e+09,2.919553e+09,2.965922e+09,3.056425e+09,0.000000e+00,0.000000e+00,0.0
1,Afghanistan,38041754,AFG,GDP (current US$),NY.GDP.MKTP.CD,5.377778e+08,5.488889e+08,5.466667e+08,7.511112e+08,8.000000e+08,...,1.780429e+10,2.000160e+10,2.056107e+10,2.048489e+10,1.990711e+10,1.801775e+10,1.886995e+10,1.835388e+10,1.929110e+10,0.0
2,Angola,31825295,AGO,GDP (current US$),NY.GDP.MKTP.CD,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,1.120000e+11,1.280000e+11,1.370000e+11,1.460000e+11,1.160000e+11,1.010000e+11,1.220000e+11,1.010000e+11,8.881570e+10,0.0
3,Albania,2854191,ALB,GDP (current US$),NY.GDP.MKTP.CD,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,1.289077e+10,1.231983e+10,1.277622e+10,1.322814e+10,1.138685e+10,1.186120e+10,1.301969e+10,1.514702e+10,1.527918e+10,0.0
4,Andorra,77142,AND,GDP (current US$),NY.GDP.MKTP.CD,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,3.629204e+09,3.188809e+09,3.193704e+09,3.271808e+09,2.789870e+09,2.896679e+09,3.000181e+09,3.218316e+09,3.154058e+09,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,Kosovo,1794248,XKX,GDP (current US$),NY.GDP.MKTP.CD,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,6.701698e+09,6.499807e+09,7.074778e+09,7.396705e+09,6.442916e+09,6.719172e+09,7.245707e+09,7.942962e+09,7.926134e+09,0.0
260,"Yemen, Rep.",29161922,YEM,GDP (current US$),NY.GDP.MKTP.CD,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,3.272642e+10,3.540134e+10,4.041524e+10,4.320647e+10,4.245062e+10,3.093598e+10,2.673614e+10,2.348627e+10,2.258108e+10,0.0
261,South Africa,58558270,ZAF,GDP (current US$),NY.GDP.MKTP.CD,7.575397e+09,7.972997e+09,8.497997e+09,9.423396e+09,1.037400e+10,...,4.160000e+11,3.960000e+11,3.670000e+11,3.510000e+11,3.180000e+11,2.960000e+11,3.500000e+11,3.680000e+11,3.510000e+11,0.0
262,Zambia,17861030,ZMB,GDP (current US$),NY.GDP.MKTP.CD,7.130000e+08,6.962857e+08,6.931429e+08,7.187143e+08,8.394286e+08,...,2.345952e+10,2.550306e+10,2.804555e+10,2.715073e+10,2.124334e+10,2.095475e+10,2.586817e+10,2.631214e+10,2.330977e+10,0.0


In [8]:
# DataFrame for SQL table country_master
country_master = gdp_clean_data[['country_name','iso_code','Population_2019','indicator_name','indicator_code','GDP_2019']]
country_master


,country_name,iso_code,Population_2019,indicator_name,indicator_code,GDP_2019
0,Aruba,ABW,106314,GDP (current US$),NY.GDP.MKTP.CD,0.000000e+00
1,Afghanistan,AFG,38041754,GDP (current US$),NY.GDP.MKTP.CD,1.929110e+10
2,Angola,AGO,31825295,GDP (current US$),NY.GDP.MKTP.CD,8.881570e+10
3,Albania,ALB,2854191,GDP (current US$),NY.GDP.MKTP.CD,1.527918e+10
4,Andorra,AND,77142,GDP (current US$),NY.GDP.MKTP.CD,3.154058e+09
...,...,...,...,...,...,...
259,Kosovo,XKX,1794248,GDP (current US$),NY.GDP.MKTP.CD,7.926134e+09
260,"Yemen, Rep.",YEM,29161922,GDP (current US$),NY.GDP.MKTP.CD,2.258108e+10
261,South Africa,ZAF,58558270,GDP (current US$),NY.GDP.MKTP.CD,3.510000e+11
262,Zambia,ZMB,17861030,GDP (current US$),NY.GDP.MKTP.CD,2.330977e+10


In [9]:
# Create the path to your file directory and variable for the file
#file_dir='C:\Users\aksha\Desktop\Class folder\Final project\my work\segment2\Resources\Cleaned_data'

# The final cleaned gdp data
gdp_cleaned_data = gdp_clean_data.to_csv('../Resources/Cleaned_data/gdp_cleaned_data.csv',index =False)
country_master = gdp_clean_data.to_csv('../Resources/Cleaned_data/country_master.csv',index =False)